In [1]:
# Parser for Passatutto data
import sys
sys.path.append('C:\\Users\\Gosia\\Desktop\\FDR-Metabolomics\\src\\')
import passatutto_parser as pp

In [2]:
# Taking all the files from MassBankOrbi (queries) and parsing them to json objects
pre_spectrums_query = pp.PassatuttoParser(r'C:\\Users\\Gosia\\Desktop\\MassbankOrbi').parse_folder()

processed 100 files
processed 200 files
processed 300 files
processed 400 files
Finished parsing of 458 spectra 


In [ ]:
# Taking all files from Gnps (library) and parsing them to json objects
pre_spectrums_lib = pp.PassatuttoParser('C:\\Users\\Gosia\\Desktop\\Gnps').parse_folder()

processed 100 files
processed 200 files
processed 300 files
processed 400 files
processed 500 files
processed 600 files
processed 700 files
processed 800 files
processed 900 files
processed 1000 files
processed 1100 files
processed 1200 files
processed 1300 files
processed 1400 files
processed 1500 files
processed 1600 files
processed 1700 files
processed 1800 files
processed 1900 files
processed 2000 files
processed 2100 files
processed 2200 files
processed 2300 files
processed 2400 files
processed 2500 files
processed 2600 files
processed 2700 files
processed 2800 files
processed 2900 files
processed 3000 files
processed 3100 files
processed 3200 files
processed 3300 files
processed 3400 files
processed 3500 files
processed 3600 files


In [ ]:
# Taking all files from decoys and parsing them to json objects
pre_spectrums_decoys_cond = pp.DecoyParserPassattuto(r'C:\\Users\\Gosia\\Desktop\\GnpsDecoyConditionalPeaks').parse_folder()
pre_spectrums_decoys_rand = pp.DecoyParserPassattuto(r'C:\\Users\\Gosia\\Desktop\\GnpsDecoyRandomPeaks').parse_folder()

In [ ]:
# Using MatchMS to create spectra for both
from matchms.importing.load_from_json import as_spectrum
spectrums_query = []
for i, s in enumerate( pre_spectrums_query ):
    spectrums_query.append(as_spectrum(s))
    if i and i % 100 == 0:
        print('processed %d' % i)
spectrums_lib = []
for i, s in enumerate( pre_spectrums_lib ):
    spectrums_lib.append(as_spectrum(s))
    if i and i % 100 == 0:
        print('processed %d' % i)

In [ ]:
# Using MatchMS to create decoy spectra
decoys_cond = []
for i, s in enumerate( pre_spectrums_decoys_cond ):
    decoys_cond.append(as_spectrum(s))
    if i and i % 100 == 0:
        print('processed %d'% i)

decoys_rand = []
for i, s in enumerate( pre_spectrums_decoys_rand ):
    decoys_rand.append(as_spectrum(s))
    if i and i % 100 == 0:
        print('processed %d'% i)


In [ ]:
# Calculating cosine similarity for target-query match
from cosine_calc import get_hits
hits = get_hits(spectrums_query, spectrums_lib, 3, cosine_tol=0.005, passatutto=True)

In [ ]:
from rdkit.Chem import MolFromSmiles, MolToSmiles, MolFromInchi
from rdkit.Chem.rdMolDescriptors import CalcExactMolWt, CalcMolFormula
def add_pmz(specs):
    for s in specs:
        s.set('precursor_mz', s.get('parent_mass', 0))

add_pmz(spectrums_query)

In [ ]:
# Calculating cosine similarity for query-decoy match
from cosine_calc import get_hits
hits_decoys_cond = get_hits(spectrums_query, decoys_cond, 3, "precursor_mz", cosine_tol=0.005, decoys=True, passatutto=True)
hits_decoys_rand = get_hits(spectrums_query, decoys_rand, 3, "precursor_mz", cosine_tol=0.005, decoys=True, passatutto=True)


In [ ]:
print(len(hits))
print(len(hits_decoys_cond))
print(len(hits_decoys_rand))

In [ ]:
# Calculating true q-value scores
from q_value_calc import calculate_q_value
q_list_true = calculate_q_value(hits)

In [ ]:
# Calculating estimated q-value scores
q_list_estimated_cond = calculate_q_value(hits+hits_decoys_cond,True)
q_list_estimated_rand = calculate_q_value(hits+hits_decoys_rand,True)

In [ ]:
import plot_q_vals

# plot estimated and true q-values
def combine_true_est(q_val_true,q_val_est):
    res = []
    q_idx = 0
    for q_e, _, score in q_val_est:
        while q_idx < len(q_val_true)-1 and q_val_true[q_idx+1][2] >= score:
            q_idx += 1
        res.append((score, q_val_true[q_idx][0], q_e))
    return res
        
scores_cond, trues_cond, estimateds_cond = zip(*combine_true_est(q_list_true, q_list_estimated_cond))
scores_rand, trues_rand, estimateds_rand = zip(*combine_true_est(q_list_true, q_list_estimated_rand))

plot_q_vals.plot_q_vals( {'spectrum-based method':(trues_cond,estimateds_cond), 'naive method':(trues_rand,estimateds_rand)} )